In [1]:
# import nltk
# nltk.download("stopwords")
# nltk.download('wordnet')

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import re
import preprocessor as p
import emoji
import codecs
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords = set(stopwords.words('english')) - set(['no', 'not'])

c:\users\pooja\appdata\local\programs\python\python37\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}

In [4]:
def doc2vec_model(X, y):
    print("Preparing tagged data...")
    tagged_data = []
    for i in tqdm(range(len(X))):
        text = X[i]
        tagged_data.append(TaggedDocument(words=word_tokenize(text), tags=[str(i)]))
    print("Preparing Vectors...")
    max_epochs = 10
    vec_size = 20
    alpha = 0.025
    model = Doc2Vec(vector_size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm=0)
    model.build_vocab(tagged_data)
    for epoch in tqdm(range(max_epochs)):
        model.train(tagged_data,total_examples=model.corpus_count, epochs=model.epochs)
        model.alpha -= 0.0002
        model.min_alpha = model.alpha
    vector_list = []
    for i in range(len(model.dv)):
        vector_list.append(model.dv[i])
    print("Training the Model...")
    X = np.array(vector_list)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("Model Training Finished.")
    return cm, accuracy, f1

In [5]:
dataset = pd.read_csv('sentiment.csv')

In [6]:
dataset.shape

(11970, 2)

In [7]:
dataset.head(5)

,Text,Sentiment
0,RT @notarealnun: Please don't vote for the guy...,1
1,OpChemtrails: Climate change models are rigged...,-1
2,RT @RichardCowley2: New Mann-made global warmi...,-1
3,"RT @goodoldcatchy: All terrorism is a threat, ...",0
4,RT @StephenSchlegel: she's thinking about how ...,1


In [8]:
dataset["Sentiment"].unique()

array([ 1, -1,  0], dtype=int64)

In [9]:
dataset["Sentiment"].value_counts()

 1    3990
-1    3990
 0    3990
Name: Sentiment, dtype: int64

In [10]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:04<00:00, 2597.24it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:11<00:00,  7.10s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[880 375  89]
 [309 719 234]
 [123 259 963]]
Accuracy:
0.6484434320425209
F1-Score:
0.6484434320425209


In [11]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.lower()
    dataset.loc[i, "Text"] = v

In [12]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:04<00:00, 2765.16it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:12<00:00,  7.26s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[874 359 111]
 [309 708 245]
 [128 271 946]]
Accuracy:
0.6398380156922299
F1-Score:
0.6398380156922299


In [13]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace('#', ' ')
    v = v.replace('@', ' ')
    v = p.clean(v)
    dataset.loc[i, "Text"] = v

In [14]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:04<00:00, 2798.07it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:10<00:00,  7.01s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[884 354 106]
 [331 687 244]
 [157 295 893]]
Accuracy:
0.6236395849152113
F1-Score:
0.6236395849152113


In [15]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace("’", "'")
    for key in contractions:
        if key in v:
            v = re.sub(key, contractions[key], v)
    dataset.loc[i, "Text"] = v

In [16]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:04<00:00, 2444.14it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:12<00:00,  7.24s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[857 378 109]
 [305 718 239]
 [143 237 965]]
Accuracy:
0.6428752214629208
F1-Score:
0.6428752214629208


In [17]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = codecs.decode(v, 'unicode_escape')
    v = emoji.demojize(v.encode('utf-16', 'surrogatepass').decode('utf-16'))
    dataset.loc[i, "Text"] = v

c:\users\pooja\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: invalid escape sequence '\ '
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:05<00:00, 2270.71it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:12<00:00,  7.23s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[899 327 118]
 [293 733 236]
 [111 253 981]]
Accuracy:
0.6613515565679575
F1-Score:
0.6613515565679575


In [19]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = re.sub("[^a-z0-9 ]", " ", v).strip()
    v = re.sub(" +", " ", v)
    dataset.loc[i, "Text"] = v

In [20]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:02<00:00, 4281.26it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:13<00:00,  7.37s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[852 375 117]
 [320 692 250]
 [162 305 878]]
Accuracy:
0.613009364717793
F1-Score:
0.613009364717793


In [21]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    temp_v = v.split()
    l = WordNetLemmatizer()
    v = [l.lemmatize(word) for word in temp_v if not word in stopwords]
    v = ' '.join(v)
    dataset.loc[i, "Text"] = v

In [22]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:04<00:00, 2577.24it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:07<00:00,  6.76s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[886 354 104]
 [294 663 305]
 [138 270 937]]
Accuracy:
0.6292077954948114
F1-Score:
0.6292077954948114


In [25]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace("’", "'")
    for key in contractions:
        if key in v:
            v = re.sub(key, contractions[key], v)
    v = codecs.decode(v, 'unicode_escape')
    v = emoji.demojize(v.encode('utf-16', 'surrogatepass').decode('utf-16'))
    dataset.loc[i, "Text"] = v

c:\users\pooja\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: invalid escape sequence '\ '
  import sys


In [26]:
cm, accuracy, f1 = doc2vec_model(dataset.loc[:, "Text"].values, dataset.loc[:, "Sentiment"].values)
print("Confusion Matrix:")
print(cm)
print("Accuracy:")
print(accuracy)
print("F1-Score:")
print(f1)

Preparing tagged data...


100%|██████████████████████████████████████████████████████████████████████████| 11970/11970 [00:02<00:00, 4284.07it/s]


Preparing Vectors...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.61s/it]

Training the Model...
Model Training Finished.
Confusion Matrix:
[[871 361 112]
 [284 748 230]
 [135 263 947]]
Accuracy:
0.6494558339660845
F1-Score:
0.6494558339660845
